# compare time of different read data methods

In [1]:
import numpy as np
import pandas as pd
import pptk
import datetime
import torch

## use numpy ndarray read data

In [4]:
file = "./data/arch/Train/1_TR_cloister.txt"
time3 = datetime.datetime.now()
data = np.loadtxt(file)
scene_points = data[:,0:3].astype('float32')
segment_label = data[:,6].astype('int64')
time4 = datetime.datetime.now()
print(scene_points.shape)
print(segment_label.shape)
print(time4-time3)

(15740229, 3)
(15740229,)
0:03:34.670069


In [19]:
import numpy as np
choice = np.random.choice(len(segment_label), 1024, replace=True)
point_set = scene_points[choice, :]

NameError: name 'segment_label' is not defined

## use numpy read file + use torch read data

In [3]:
file = "./data/arch/Train/1_TR_cloister.txt"
time1 = datetime.datetime.now()
data = np.loadtxt(file)
scene_points = torch.from_numpy(data[:, 0:3].astype('float32'))
segment_label = torch.from_numpy(data[:,6].astype('int32'))
time2 = datetime.datetime.now()
print(scene_points.shape)
print(segment_label.shape)
print(time2-time1)

torch.Size([15740229, 3])
torch.Size([15740229])
0:03:52.921357


## Test Block - random sample

In [1]:
range(2048)

range(0, 2048)

In [29]:
def sample_data(data, num_sample):
    """ data is in N x ...
        we want to keep num_samplexC of them.
        if N > num_sample, we will randomly keep num_sample of them.
        if N < num_sample, we will randomly duplicate samples.
    """
    N = data.shape[0]
    if (N == num_sample):
        return data, range(N)
    elif (N > num_sample):
        sample = np.random.choice(N, num_sample)
        return data[sample, ...], sample
    else:
        sample = np.random.choice(N, num_sample-N)
        dup_data = data[sample, ...]
        return np.concatenate([data, dup_data], 0), list(range(N))+list(sample)

In [9]:
def sample_data_label(data, label, num_sample):
    new_data, sample_indices = sample_data(data, num_sample)
    new_label = label[sample_indices]
    return new_data, new_label

In [5]:
import numpy as np
data = np.random.random((2080,3))
data

array([[0.35296084, 0.72715114, 0.40660786],
       [0.79695103, 0.35178304, 0.1483732 ],
       [0.02184964, 0.89369391, 0.21384761],
       ...,
       [0.72761692, 0.63170081, 0.70812537],
       [0.58366183, 0.69593253, 0.49909007],
       [0.46816175, 0.91908899, 0.4233715 ]])

In [11]:
label = np.random.randint(0,10,size=[2080,1])
label

array([[2],
       [1],
       [0],
       ...,
       [3],
       [5],
       [7]])

In [13]:
block_data_sampled, block_label_sampled = sample_data_label(data, label, 2048)

In [18]:
print(block_data_sampled.shape)
print(block_label_sampled.shape)

(2048, 3)
(2048, 1)


In [19]:
data2 = np.random.random((2000,3))
data2

array([[0.364098  , 0.30908428, 0.63440443],
       [0.30548676, 0.40795002, 0.67965822],
       [0.52483172, 0.76652738, 0.86369489],
       ...,
       [0.16839643, 0.47371034, 0.72023846],
       [0.05214054, 0.68377316, 0.61206864],
       [0.47624035, 0.64036852, 0.56606007]])

In [20]:
label2 = np.random.randint(0,10,size=[2000,1])

In [30]:
block_data2_sampled, block_label2_sampled = sample_data_label(data2, label2, 2048)

In [31]:
print(block_data2_sampled.shape)
print(block_label2_sampled.shape)

(2048, 3)
(2048, 1)


In [40]:
current_data = np.tile(block_data2_sampled, (93,1,1))
current_data.shape

(93, 2048, 3)

In [46]:
file_size = current_data.shape[0]
batch_size = 8
num_batches = file_size//batch_size
all_data = []

print(num_batches)
for batch_idx in range(num_batches):
    #if num_batches == file_size
    start_idx = batch_idx*8
    end_idx = (batch_idx+1)*8
    all_data.append(current_data[start_idx:end_idx, :, :])
all_data = np.array(all_data)
print(all_data.shape)

11
(11, 8, 2048, 3)


## Test block - gen batch to hdf5 file

In [53]:
NUM_POINT = 2048
H5_BATCH_SIZE = 1000
data_dim = [NUM_POINT, 3]
label_dim = [NUM_POINT]

batch_data_dim = [H5_BATCH_SIZE] + data_dim
batch_label_dim = [H5_BATCH_SIZE] + label_dim
h5_batch_data = np.zeros(batch_data_dim, dtype = np.float32)
h5_batch_label = np.zeros(batch_label_dim, dtype = np.uint8)
buffer_size = 0  # state: record how many samples are currently in buffer
h5_index = 0 # state: the next h5 file to save

print([H5_BATCH_SIZE])
print(batch_data_dim)
print(h5_batch_data.shape)
print(h5_batch_label.shape)

[1000]
[1000, 2048, 3]
(1000, 2048, 3)
(1000, 2048)


In [54]:
data_size = current_data.shape[0]
h5_batch_data[buffer_size:buffer_size+data_size, ...] = current_data
buffer_size += data_size

print(h5_batch_data.shape)

(1000, 2048, 3)


In [55]:
h5_batch_data

array([[[0.36409798, 0.3090843 , 0.6344044 ],
        [0.30548677, 0.40795   , 0.67965823],
        [0.5248317 , 0.76652735, 0.8636949 ],
        ...,
        [0.20645209, 0.01392364, 0.14218394],
        [0.22136463, 0.86750555, 0.24099368],
        [0.46171084, 0.4232767 , 0.58267117]],

       [[0.36409798, 0.3090843 , 0.6344044 ],
        [0.30548677, 0.40795   , 0.67965823],
        [0.5248317 , 0.76652735, 0.8636949 ],
        ...,
        [0.20645209, 0.01392364, 0.14218394],
        [0.22136463, 0.86750555, 0.24099368],
        [0.46171084, 0.4232767 , 0.58267117]],

       [[0.36409798, 0.3090843 , 0.6344044 ],
        [0.30548677, 0.40795   , 0.67965823],
        [0.5248317 , 0.76652735, 0.8636949 ],
        ...,
        [0.20645209, 0.01392364, 0.14218394],
        [0.22136463, 0.86750555, 0.24099368],
        [0.46171084, 0.4232767 , 0.58267117]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0

## Generate hdf5 format (1000, 2048, 10) point clouds

In [2]:
!ls

 common.py  'experiment updating.md'   __pycache__   utils
 data	     LOG		       README.md     visualization.ipynb
 datasets    model		       sampling      visualization.py


In [3]:
cd datasets

/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/datasets


In [4]:
!ls

ArCH.py        gen_arch_h5.py  ShapeNetCore.py
dataloader.py  __pycache__     synsetoffset2category.txt


In [12]:
!python gen_arch_h5.py

input file: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/1_TR_cloister.txt
input scene: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/1_TR_cloister.txt and shape: (15740229, 7)
block number: 77, 38
block data list size: 693
(693, 2048, 6), (693, 2048)
output file size: (693, 2048, 6), (693, 2048)
sample number now: 693now insert_batch
now in enough space location, store data in memory
finish 0 times
input file: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/15_OTT_church.txt
input scene: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/15_OTT_church.txt and shape: (13302903, 7)
block number: 82, 118
block data list size: 708
(708, 2048, 6), (708, 2048)
output file size: (708, 2048, 6), (708, 2048)
sample number now: 1401now insert_batch
Stored /home/yw/Documents/experiment/Unsupervise

In [25]:
import numpy as np
block_size=1.0
stride = 1.0
data_label_filename = "/home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/14_TRE_square.txt"
data_label = np.loadtxt(data_label_filename)
data = data_label[:,0:6]
#get the corner location for our sampling blocks
limit = np.amax(data,0)[0:3]

#calculate number of blocks and add into block list
xbeg_list = []
ybeg_list = []

num_block_x = int(np.ceil((limit[0] - block_size) / stride)) + 1
num_block_y = int(np.ceil((limit[1] - block_size) / stride)) + 1

In [26]:
num_block_x, num_block_y

(62, 78)

In [27]:
!python gen_arch_h5.py

input file: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/2_TR_church.txt
input scene: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/2_TR_church.txt and shape: (20862139, 7)
block number: 93, 46
block data list size: 641
(641, 2048, 6), (641, 2048)
output file size: (641, 2048, 6), (641, 2048)
sample number now: 641now insert_batch
now in enough space location, store data in memory
finish 0 times
input file: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/9_SMV_chapel_10.txt
input scene: /home/yw/Documents/experiment/Unsupervised-learning-on-LoD3-building-point-cloud/data/arch/Train1/9_SMV_chapel_10.txt and shape: (2193189, 7)
block number: 51, 62
block data list size: 110
(110, 2048, 6), (110, 2048)
output file size: (110, 2048, 6), (110, 2048)
sample number now: 751now insert_batch
now in enough space location, store data in memory


## test PointCNN point clouds split algorithm

In [5]:
import os
import sys
import math
import numpy as np
from datetime import datetime

filename_txt = "./data/arch/Test1/A_SMG_portico.txt"
print('{}-Loading {}...'.format(datetime.now(), filename_txt))
all_data = np.loadtxt(filename_txt)
xyzrgbl = all_data[:, 0:7]
labels = xyzrgbl[:,-1]

xyz = xyzrgbl[:,0:3]
rgb = xyzrgbl[:,3:6] / 255 - 0.5
block_size = 2.0
offsets = [('zero', 0.0), ('half', block_size / 2)]
print(offsets)
for offset_name, offset in offsets:
    print('{}-Computing block id of {} points...'.format(datetime.now(), xyzrgbl.shape[0]))
    xyz_min = np.amin(xyz, axis=0, keepdims=True) - offset
    xyz_max = np.amax(xyz, axis=0, keepdims=True)
    print("xyz_min: " + str(xyz_min))
    print("xyz_max: " + str(xyz_max))
    block_size = (block_size, block_size, 2 * (xyz_max[0, -1] - xyz_min[0, -1]))
    print(xyz_max[0, -1] - xyz_min[0, -1])
    print("block_size: " + str(block_size))
    xyz_blocks = np.floor((xyz - xyz_min) / block_size).astype(np.int)
    print(xyz_blocks)
    print('{}-Collecting points belong to each block...'.format(datetime.now(), xyzrgbl.shape[0]))
    blocks, point_block_indices, block_point_counts = np.unique(xyz_blocks, return_inverse=True, return_counts=True, axis=0)
    print(blocks)
    print(point_block_indices)
    print(block_point_counts)
    block_point_indices = np.split(np.argsort(point_block_indices), np.cumsum(block_point_counts[:-1]))
    print('{}-{} is split into {} blocks.'.format(datetime.now(), dataset, blocks.shape[0]))

2020-10-28 17:20:54.410138-Loading ./data/arch/Test1/A_SMG_portico.txt...
[('zero', 0.0), ('half', 1.0)]
2020-10-28 17:24:46.661784-Computing block id of 17798012 points...
xyz_min: [[ 69.39041138 103.79975128 374.05963135]]
xyz_max: [[133.0887146  156.79910278 385.18084717]]
11.121215819999975
block_size: (2.0, 2.0, 22.24243163999995)
[[29 22  0]
 [29 22  0]
 [29 22  0]
 ...
 [30 24  0]
 [30 24  0]
 [30 24  0]]
2020-10-28 17:24:48.781681-Collecting points belong to each block...
[[ 0  3  0]
 [ 0  4  0]
 [ 0  5  0]
 [ 0  6  0]
 [ 1  2  0]
 [ 1  3  0]
 [ 1  4  0]
 [ 1  5  0]
 [ 1  6  0]
 [ 1  7  0]
 [ 2  2  0]
 [ 2  3  0]
 [ 2  4  0]
 [ 2  5  0]
 [ 2  6  0]
 [ 2  7  0]
 [ 2  8  0]
 [ 2  9  0]
 [ 3  1  0]
 [ 3  2  0]
 [ 3  3  0]
 [ 3  4  0]
 [ 3  5  0]
 [ 3  6  0]
 [ 3  7  0]
 [ 3  8  0]
 [ 3  9  0]
 [ 3 10  0]
 [ 4  0  0]
 [ 4  1  0]
 [ 4  2  0]
 [ 4  3  0]
 [ 4  4  0]
 [ 4  5  0]
 [ 4  6  0]
 [ 4  7  0]
 [ 4  8  0]
 [ 4  9  0]
 [ 4 10  0]
 [ 4 11  0]
 [ 4 12  0]
 [ 5  0  0]
 [ 5  1  0]

NameError: name 'dataset' is not defined

In [6]:
blocks.shape[0]

259

In [10]:
def load_h5(h5_filename):
    f = h5py.File(h5_filename, 'r')
    data = f['data'][:]
    label = f['label'][:]
    data_num = f['data_num'][...].astype(np.int32)
    labels_seg = f['label_seg'][...].astype(np.int64)
    if 'indices_split_to_full' in f:
        indices_split_to_full = f['indices_split_to_full'][...].astype(np.int64)
    return (data, label, data_num, labels_seg, indices_split_to_full)

In [11]:
import h5py
data_train, _, data_num_train, label_train, _ = load_h5("./data/arch/Train1/12_KAS_pavillion_1.txt_8192_half_0.h5")
print(data_train.shape)
print(label_train.shape)
print(data_num_train.shape)

(96, 8196, 6)
(96, 8196)
(96,)


In [20]:
def load_seg(filelist):
    points = []
    labels = []
    point_nums = []
    labels_seg = []
    indices_split_to_full = []

    for fn in filelist:
        data = h5py.File(fn, 'r')
        points.append(data['data'][...].astype(np.float32))
        labels.append(data['label'][...].astype(np.int64))
        point_nums.append(data['data_num'][...].astype(np.int32))
        labels_seg.append(data['label_seg'][...].astype(np.int64))
        if 'indices_split_to_full' in data:
            indices_split_to_full.append(data['indices_split_to_full'][...].astype(np.int64))

    return (np.concatenate(points, axis=0),
            np.concatenate(labels, axis=0),
            np.concatenate(point_nums, axis=0),
            np.concatenate(labels_seg, axis=0),
            np.concatenate(indices_split_to_full, axis=0) if indices_split_to_full else None)

In [21]:
filelist = ["data/arch/Train1/12_KAS_pavillion_1.txt_8192_half_0.h5","data/arch/Train1/12_KAS_pavillion_1.txt_8192_zero_0.h5"]
data_train, _, data_num_train, label_train, _ = load_seg(filelist)
print(data_train.shape)
print(label_train.shape)
print(data_num_train.shape)

(191, 8196, 6)
(191, 8196)
(191,)


In [16]:
# Prepare inputs
import os.path
import sys
from datetime import datetime
sys.path.append(os.path.join(".", 'utils'))
import pc_utils


filelist = "./data/arch/train_data_files.txt"
batch_size = 12
num_epochs = 256
print('{}-Preparing datasets...'.format(datetime.now()))
is_list_of_h5_list = not pc_utils.is_h5_list(filelist)
if is_list_of_h5_list:
    seg_list = pc_utils.load_seg_list(filelist)
    seg_list_idx = 0
    filelist_train = seg_list[seg_list_idx]
    seg_list_idx = seg_list_idx + 1
else:
    filelist_train =  filelist
print(filelist_train)
data_train, _, data_num_train, label_train, _ = pc_utils.load_seg(filelist_train)
print(data_train.shape)
print(data_num_train.shape)
print(label_train.shape)
num_train = data_train.shape[0]
point_num = data_train.shape[1]

print('{}-{:d} training samples.'.format(datetime.now(), num_train))
batch_num = (num_train * num_epochs + batch_size - 1) // batch_size
print('{}-{:d} training batches.'.format(datetime.now(), batch_num))

2020-10-29 17:36:45.085651-Preparing datasets...
./data/arch/./filelists/train_files_g_0.txt
(8192, 2048, 6)
(8192,)
(8192, 2048)
2020-10-29 17:36:45.708164-8192 training samples.
2020-10-29 17:36:45.708325-174763 training batches.


## !python ./datasets/dataloader.py